In [20]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [21]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter/special_functions.pyc'>

### Make fit

In [22]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 0, # max 10
        'exclude' : [] # put LECs here
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : True,
    
    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400', u'a15m130', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

In [23]:
nnlo_lecs = ['A_a', 'A_k', 'A_p']
for j in range(len(nnlo_lecs)):
    p_dict['order']['exclude'] = nnlo_lecs[:j] + nnlo_lecs[j+1:]
    
    t0 = time.time()

    # Load data
    data_loader = dl.data_loader()
    fit_data = data_loader.get_fit_data()

    # Get prior
    prior = None
    if p_dict['use_prior']:
        prior = data_loader.get_prior(p_dict['fit_type'])

    # Make bootstrapper
    bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                                   fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

    if p_dict['make_plots']:
        data_loader.save_plots(
            bootstrapper.make_plots(
                show_error_ellipses=p_dict['show_error_ellipses'], 
                show_bootstrap_histograms=p_dict['show_bs_histograms']),
            output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
        )
    else:
        print bootstrapper

    if p_dict['save_prior']:
        print 'Saving prior...'
        data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

    if p_dict['save_results']:
        data_loader.save_fit_info(bootstrapper.get_fit_info())


    t1 = time.time()

    print "\nTotal time (s): ", t1 - t0, "\n"

Making fits...
100% complete Time (s):  1.09921121597
Compiling results...


Fit type: ma (F^2 = FKFpi, bsN = 1)

Fitting to nnlo 
 with volume corrections O(0) 
 chained: True 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections):

F_K / F_pi = 1.1940(42) [1.1932(19)]   (delta_su2 = -0.00235(24))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
      L_4 0          -0.000333 (12)            1               0.0 (3.1)
          1             0.0000 (10)        L_5 0           0.000795 (67)
      A_a 0               6.2 (2.5)            1             0.0000 (10)
---
boot0 fit results:
Least Square Fit:
  chi2/dof [dof] = 2.6 [16]    Q = 0.00037    logGBF = 53.766

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08*,1e-10,1e-10)    (itns/time = 5/0.3)
  fitter = scipy_least_squares    method = trf


Total time (s):  1.75135087967 

Making fits...
100% complete Time (s):  0